In [5]:
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import edhec_risk_kit as erk
import pandas as pd
import numpy as np
import config
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
all_data = erk.get_df(filename=config.fileName,format=config.dateFormat, mode='return', to_per=True)
all_data.columns = all_data.columns.str.replace(' ', '_')

In [20]:
plot_data = {}
plot_data.update(Real_Estate=all_data['Real_Estate'])

In [21]:
wealth_data_plots, risk_info = erk.get_wealth_index_risk(plot_data)

In [22]:
pyo.iplot(wealth_data_plots)
risk_info

,Real_Estate
ann_ret,0.001106
ann_vol,0.002806
sharpe_ratio,-10.297409
semi_dev,0.000718
Kurtosis,9.107918
Skew,-0.201885
VaR,-0.001181
CVaR,-0.002395
Drawdown,-0.007561


In [27]:
explanatory_df = all_data[config.factorName]
dependent_df = all_data[config.asset_class]
regression_result = erk.regress(dependent_df, explanatory_df, excess_mkt=False, rfcol=config.rfcol)

In [28]:
regression_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Real_Estate   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     59.99
Date:                Thu, 30 Jul 2020   Prob (F-statistic):           1.63e-46
Time:                        14:56:26   Log-Likelihood:                 2580.8
No. Observations:                 405   AIC:                            -5150.
Df Residuals:                     399   BIC:                            -5126.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
World_Equities           0.3103      0.051      6.036      0.000       0.209       0.411
US_Treasuries            1.2125      0.121     10.027      0.000       0.975       1.450
Bond_Risk_Premium        1.0542      0.103     10.208      0.000       0.851       1.257
Inflation_Protection     0.2879      0.129      2.229      0.026       0.034       0.542
Currency_Protection      0.3618      0.173      2.090      0.037       0.021       0.702
Alpha                -3.587e-05   2.28e-05     -1.575      0.116   -8.06e-05     8.9e-06
==============================================================================
Omnibus:                       81.133   Durbin-Watson:                   2.289
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1295.204
Skew:                           0.254   Prob(JB):                    5.63e-282
Kurtosis:                      11.746   Cond. No.                     8.43e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""